In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np
from tqdm import tqdm
import time
np.random.seed(42)

In [ ]:
class Individual:
    
    def __init__(self, n: int):
        self.genes = np.random.randint(low=0, high=2, size=n)
        self.fitness = -np.inf
        
    def __str__(self):
        return f'(Ind: {self.genes}, {self.fitness})'
    
    def __repr__(self):
        return str(self)

In [ ]:
ind = Individual(10)
ind

In [ ]:
def onemax(i: Individual):
    return np.sum(i.genes)  

ind.fitness = onemax(ind)
ind

In [ ]:
def evaluate(ind: Individual, objective):
    ind.fitness = objective(ind)

evaluate(ind, onemax)
ind

In [ ]:
def mutate(ind: Individual, mutation_rate=0.05):
    new_genes = np.copy(ind.genes)
    for i in range(len(new_genes)):
        if np.random.rand() < mutation_rate:
            new_genes[i] = not ind.genes[i]
    child = Individual(len(ind.genes))
    child.genes = new_genes
    return child

<div class="alert alert-success">
    <b>Exercise</b>
    <br/>
    Run the (1+1) EA on the Leading Ones problem. Does it converge near $0(n \log n)$, or $O(n^2)$?
    <br/>
</div>

In [ ]:
def evaluate(ind: Individual, objective):
    ind.fitness = objective(ind)

#evaluate(ind, leading_ones)
#ind

In [ ]:
def leading_ones(ind: Individual):
    f = 0
    for i in range(len(ind.genes)):
        if not ind.genes[i]:
            f = i
            break
    return f
#print(ind)
#leading_ones(ind)

In [ ]:
def one_plus_one_leading_ones(ind_length: int, num_generations: int, objective):
    fits = np.zeros(num_generations)
    parent = Individual(ind_length)
    evaluate(parent, objective)
    
    for i in tqdm(range(len(fits))):
        child = mutate(parent, 1.0/len(parent.genes))
        evaluate(child, objective)

        if child.fitness >= parent.fitness:
            parent = child
            
        fits[i] = parent.fitness
    return fits

In [ ]:
np.random.seed(int(time.time()))
n = 100
num_generations = int(np.round(n * n))
fits = one_plus_one_leading_ones(n, num_generations, leading_ones)
print(fits[-1])  
plt.plot(fits, 'b')
plt.xlabel("Generations")
plt.ylabel("Fitness")
plt.title(f"leading_ones on {n} Dimensions");


Convergence occurs in n * n

In [ ]:
n = 100
λ = int(round(np.log2(n)*np.log2(np.log2(n))/np.log2(np.log2(np.log2(n)))))
print(λ)

<div class="alert alert-success">
    <b>Exercise</b>
    <br/>
    Compare the (1+1) EA and $(1+\lambda)$ EA on the Leading Ones problem.
    How does the problem size (n) affect comparison of these 2 algorithms?
    <br/>
</div>

In [ ]:
n = 100
λ = int(round(np.log2(n)*np.log2(np.log2(n))/np.log2(np.log2(np.log2(n)))))
print(λ)

In [ ]:
def evaluate(ind: Individual, objective):
    ind.fitness = objective(ind)

#evaluate(ind, leading_ones)
#ind

In [ ]:
def leading_ones(ind: Individual):
    f = 0
    for i in range(len(ind.genes)):
        if not ind.genes[i]:
            f = i
            break
    return f
#print(ind)
#leading_ones(ind)

In [ ]:
 def one_plus_one_leading_ones(ind_length: int, num_generations: int, objective):
    fits = np.zeros(num_generations)
    parent = Individual(ind_length)
    #leading_ones(parent, objective)
    evaluate(parent, objective)
    
    for i in tqdm(range(len(fits))):
        child = mutate(parent, 1.0/len(parent.genes))
        evaluate(child, objective)
        #leading_ones(child, objective)
        if child.fitness >= parent.fitness:
            parent = child
            
        fits[i] = parent.fitness
    return fits

In [ ]:
def one_plus_lambda_leading_ones(ind_length: int, num_generations: int, objective, λ: int):
    fits = np.zeros(num_generations)
    expert = Individual(ind_length)
    evaluate(expert, objective)
    
    for i in tqdm(range(len(fits))):
        population = [expert,]
        best = 0
        for j in range(1, λ):
            population.append(mutate(expert))
            evaluate(population[j], objective) 
            if population[j].fitness > population[best].fitness:
                best = j
        if population[best].fitness >= expert.fitness: #survivor selection
            expert = population[best]
        fits[i] = expert.fitness
    return fits

In [ ]:
n=100
n_trials = 10
n_gens = 30000

fits_1 = np.zeros((n_gens, n_trials))
fits_𝜆 = np.zeros((int(n_gens/𝜆), n_trials))

for i in range(n_trials):
    fits_1[:, i] = one_plus_one_leading_ones(n, n_gens, leading_ones)
    fits_𝜆[:, i] = one_plus_lambda_leading_ones(n, int(n_gens/𝜆), leading_ones, 𝜆)  
    μ_1 = np.mean(fits_1, axis=1)
σ_1 = np.std(fits_1, axis=1)
μ_𝜆 = np.mean(fits_𝜆, axis=1)
σ_𝜆 = np.std(fits_𝜆, axis=1)
print(np.shape(μ_1), np.shape(σ_1))
print(np.shape(μ_𝜆), np.shape(σ_𝜆))  


# plt.plot(μ_1, label="1+1", color='b')
# plt.fill_between(np.arange(0, n_gens), μ_1+σ_1, μ_1-σ_1, facecolor='b', alpha=0.5)
# plt.plot(np.arange(1,n_gens,𝜆), μ_𝜆, label="1+lambda", color='g')
# plt.fill_between(np.arange(0, n_gens, 𝜆), μ_𝜆+σ_𝜆, μ_𝜆-σ_𝜆, facecolor='g', alpha=0.5)
# plt.xlabel("Evaluations")
# plt.ylabel("Fitness")
# plt.title(f"leading_ones on {n} Dimensions")
# plt.legend();

In [ ]:
plt.subplot(1, 2, 1)  
plt.plot(μ_1, label="1+1", color='b')
plt.title('1+1')
plt.xlabel("Evaluations")
plt.ylabel("Fitness")
plt.title(f"leading_ones on {n} Dimensions")

plt.subplot(1, 2, 2)  
plt.plot(μ_𝜆, label="1+lambda", color='g')
plt.title('1+lambda')

plt.xlabel("Evaluations")
plt.ylabel("Fitness")
plt.title(f"leading_ones on {n} Dimensions")

plt.show()

In [ ]:
n=50
n_trials = 10
n_gens = 10000

fits_1 = np.zeros((n_gens, n_trials))
fits_𝜆 = np.zeros((int(n_gens/𝜆), n_trials))

for i in range(n_trials):
    fits_1[:, i] = one_plus_one_leading_ones(n, n_gens, leading_ones)
    fits_𝜆[:, i] = one_plus_lambda_leading_ones(n, int(n_gens/𝜆), leading_ones, 𝜆)  
μ_1 = np.mean(fits_1, axis=1)
σ_1 = np.std(fits_1, axis=1)
μ_𝜆 = np.mean(fits_𝜆, axis=1)
σ_𝜆 = np.std(fits_𝜆, axis=1)
print(np.shape(μ_1), np.shape(σ_1))
print(np.shape(μ_𝜆), np.shape(σ_𝜆))  


In [ ]:
plt.subplot(1, 2, 1)  
plt.plot(μ_1, label="1+1", color='b')
plt.title('1+1')
plt.xlabel("Evaluations")
plt.ylabel("Fitness")
plt.title(f"leading_ones on {n} Dimensions")

plt.subplot(1, 2, 2)  
plt.plot(μ_𝜆, label="1+lambda", color='g')
plt.title('1+lambda')

plt.xlabel("Evaluations")
plt.ylabel("Fitness")
plt.title(f"leading_ones on {n} Dimensions")

plt.show()

In [ ]:
n=200
n_trials = 10
n_gens = 30000

fits_1 = np.zeros((n_gens, n_trials))
fits_𝜆 = np.zeros((int(n_gens/𝜆), n_trials))

for i in range(n_trials):
    fits_1[:, i] = one_plus_one_leading_ones(n, n_gens, leading_ones)
    fits_𝜆[:, i] = one_plus_lambda_leading_ones(n, int(n_gens/𝜆), leading_ones, 𝜆)  
μ_1 = np.mean(fits_1, axis=1)
σ_1 = np.std(fits_1, axis=1)
μ_𝜆 = np.mean(fits_𝜆, axis=1)
σ_𝜆 = np.std(fits_𝜆, axis=1)
print(np.shape(μ_1), np.shape(σ_1))
print(np.shape(μ_𝜆), np.shape(σ_𝜆))  


In [ ]:
plt.subplot(1, 2, 1)  
plt.plot(μ_1, label="1+1", color='b')
plt.title('1+1')
plt.xlabel("Evaluations")
plt.ylabel("Fitness")
plt.title(f"leading_ones on {n} Dimensions")

plt.subplot(1, 2, 2)  
plt.plot(μ_𝜆, label="1+lambda", color='g')
plt.title('1+lambda')

plt.xlabel("Evaluations")
plt.ylabel("Fitness")
plt.title(f"leading_ones on {n} Dimensions")

plt.show()

(1+1) EA's computational complexity increases at a slower rate because it produces only one offspring per generation, while the complexity of (1+𝜆) EA grows with 𝜆, potentially resulting in longer computation times for larger problem sizes (n).

In the context of the Leading Ones problem, which revolves around discovering highly ordered sequences of ones, (1+1) EA might face difficulties in escaping local optima as n becomes larger. Conversely, (1+𝜆) EA, with its population-based approach, stands a better chance of breaking free from local optima and uncovering longer sequences of leading ones.